In [1]:
from scipy.stats import norm
from scipy.stats import beta
from scipy.stats import gamma
from scipy.stats import chi2
from scipy.stats import gennorm

import time as time
import csv
import pickle
import datetime

import autograd.numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# import tensorflow_probability as tfp #to sample from a generalized normal distribution

from confint import confint

## experiment setting parameters

In [2]:
n = 100
dist = "gamma"
# dist = "gaussian"
# dist = "chisq"
# df = 3
# dist = "uniform"
# dist = "gennorm"

alpha = 0.1 #confidence level 1 - alpha

thread_num = 4 #number of threads #64, 40, 13 for n=10000, 1000, and 100 

## algorithm parameters

In [3]:
if n == 100:
    opt_pts_ratio = 1
else:
    opt_pts_ratio = 0.3

if dist == "uniform":
    kappa = 4.0
else:
    kappa = 8.0

if dist == "gamma":
    M = 7.0
elif dist == "gaussian":
    M = 12.0
elif dist == "uniform":
    M = 5.0
elif dist == "chisq" or dist == "gennorm":
    M = 8.0
else:
    M = 10.0

if dist == "gaussian":
    tau_max = 1e5
else:
    tau_max = 1e3
    
max_iters = 50
min_iters = 15

## run experiment

In [4]:
repeat_times = 2

results_to_save = {}
is_save = True

run_time_ave = 0
num_nan_repeat = 0
num_acc_nan_repeat = 0

now = datetime.datetime.now()
mmddyyhhmm = ("%d_%d_%d_%d_%d" % (now.month, now.day, now.year, now.hour, now.minute))
part_of_out_fn = dist + "_n_" + str(n) + "_uid_" + mmddyyhhmm

np.random.seed(1)

In [5]:
for repeat in range(repeat_times):
    if (dist == "gaussian"):
        X = np.random.randn(n)
    elif (dist == "gamma"):
        X = np.random.gamma(shape=1.0, size=n)
    elif (dist == "chisq"):
        X = np.random.chisquare(df=df, size=n)
    elif(dist == "uniform"):
        X = np.random.uniform(low=-10, high=10, size=n)
    elif(dist == "mixture"):
        n_minus1 = n / 2
        n_plus1 = n - n_minus1
        X = np.concatenate([-2 + np.random.randn(n_minus1), 2 + np.random.randn(n_plus1)])
    elif (dist == "gennorm"):
        tfd = tfp.distributions
        tf_dist = tfd.GeneralizedNormal(loc=0, scale=1.0, power=4.0)
        X = tf_dist.sample(sample_shape=(n), seed=repeat).numpy()
    else:
        print("ERROR: unsupported distribution")

    X = np.sort(X)

    conf_int = confint(X, alpha, opt_pts_ratio=opt_pts_ratio)
    t0 = time.time()
    conf_int.compute_pw_conf_ints(thread_num=thread_num, M=M, tau_max=tau_max, kappa=kappa, 
                                  max_iters=max_iters, min_iters=min_iters, verbose=False)
    
    t1 = time.time()
    
    run_time_ave += (t1 - t0) / repeat_times
    num_acc_nan_repeat += conf_int.num_nans
    if conf_int.num_nans != 0:
        num_nan_repeat += 1
    
    if len(conf_int.idxes_of_design_pts_to_opt) <= 1:
        continue

    results_to_save[dist + str(repeat) + "_sample_pts"] = X
    results_to_save[dist + str(repeat) + "_design_pts"] = X[conf_int.idxes_of_design_pts]
    results_to_save[dist + str(repeat) + "_opt_design_pts"] = conf_int.opt_pts
    results_to_save[dist + str(repeat) + "_lo"] = conf_int.low_opt_pts
    results_to_save[dist + str(repeat) + "_hi"] = conf_int.high_opt_pts
#     results_to_save[dist + str(repeat) + "_improved_lo"] = conf_int.improved_lo_opt_pts
    
    results_to_save[dist + str(repeat) + "_extended_x"] = conf_int.opt_int_pts
    results_to_save[dist + str(repeat) + "_extended_lo"] = conf_int.low_opt_int_pts
    results_to_save[dist + str(repeat) + "_extended_hi"] = conf_int.high_opt_int_pts
    
    results_to_save[dist + str(repeat) + "_run_time"] = t1 - t0
    results_to_save[dist + str(repeat) + "_failure_num"] = conf_int.num_nans
    results_to_save[dist + str(repeat) + "_failure_design_pts"] = conf_int.failure_design_pts
    
    print("trial", repeat, "run_time", t1 - t0, "failure_num", conf_int.num_nans, "failure_design_pts", conf_int.failure_design_pts)
    

num design points to optimize 13


/Users/xinyueshen/anaconda3/lib/python3.7/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/xinyueshen/anaconda3/lib/python3.7/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/xinyueshen/anaconda3/lib/python3.7/site-packages/cvxpy/

trial 0 run_time 7.26321005821228 failure_num 0 failure_design_pts []
num design points to optimize 13


/Users/xinyueshen/anaconda3/lib/python3.7/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/xinyueshen/anaconda3/lib/python3.7/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)
/Users/xinyueshen/anaconda3/lib/python3.7/site-packages/cvxpy/

trial 1 run_time 9.240768909454346 failure_num 5 failure_design_pts [0.11372219 0.2149699  1.02015481 1.64561029 2.39180054]


In [6]:
print("number of total nans", num_acc_nan_repeat)
print("number of repetitions containing nans", num_nan_repeat)
print("ave run time", run_time_ave)

number of total nans 5
number of repetitions containing nans 1
ave run time 8.251989483833313


In [7]:
ff = "data_revision1/coverage_90/pkl/" # directory to save file
if is_save:
    pickle.dump(results_to_save, open("data_%s.pkl" % (part_of_out_fn), "wb"))